In [ ]:
__Author__ = 'Freddie (Zheyuan) Fan', 'freddie.fan0120@gmail.com'

In [1]:
import quandl
from datetime import date
from scipy.stats import norm
import math
import numpy as np
import numba as nb
import pandas as pd
import ipywidgets as wid
from IPython.display import display, clear_output
from IPython.core.display import HTML
import logging
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure
import bokeh.models as bom
import seaborn as sns
from tqdm import tqdm_notebook
from collections import namedtuple
from energy_backtest import *
output_notebook()

Loading BokehJS ...

In [11]:
# Config setting
Month_code = {1: 'F', 2: 'G', 3: 'H', 4: 'J', 5: 'K', 6: 'M', 7: 'N', 8: 'Q', 9: 'U', 10: 'V', 11: 'X', 12: 'Z'}
option_multiplier = {'LO': 1, 'LN': 1}
future_multiplier = {'CL': 1000, 'NG': 10000}
option_tick_size = {'LO': 0.01, 'LN': 0.001}
future_tick_size = {'CL': 0.01, 'NG': 0.001}
cost_percentage = 0.5
size = 100
quandl.ApiConfig.api_key = 'FYVALwRtpJkfmtyysPTR'

In [109]:
def summarize_bts(bts):
    """
    summarize trade/ position/ pnl information by days.
    """
    if isinstance(bts, pd.DataFrame):
        bts['ts'] = bts.index
        bts = [bts]
    if isinstance(bts, dict):
        for k in bts:
            bts[k]['ts'] = bts[k].index
        bts = list(bts.values())
    
    bts2 = []
    for i, bt in enumerate(bts):
        bt = bt.copy()
        for c in ['LO_option_pnl', 'LN_option_pnl', 'CL_future_pnl', 'NG_future_pnl', 'LO_pnl', 'LN_pnl', 'Total_pnl']:
            bt[c] = bt[c].diff().fillna(0)
        bts2.append(bt)

    bts = pd.concat(bts2, ignore_index=True)

    bts['Option_trades'] = (bts.LO_option_trade != 0).astype('int64')
    bts['Future_trades'] = ((bts.CL_future_trade != 0) | (bts.NG_future_trade != 0)).astype('int64')
    bts['LO_option_contracts'] = bts.LO_option_trade.abs()
    bts['LN_option_contracts'] = bts.LN_option_trade.abs()
    bts['CL_future_contracts'] = bts.CL_future_trade.abs()
    bts['NG_future_contracts'] = bts.NG_future_trade.abs()

    summ = (bts.groupby('ts')
            [['LO_option_pnl', 'LN_option_pnl', 'CL_future_pnl', 'NG_future_pnl', 'LO_pnl', 'LN_pnl', 'Total_pnl',
              'LO_option_contracts', 'LN_option_contracts', 'CL_future_contracts', 'NG_future_contracts',
              'LO_option_delta', 'LO_option_gamma', 'LO_option_theta', 'LO_option_vega', 
              'LN_option_delta', 'LN_option_gamma', 'LN_option_theta', 'LN_option_vega',
              'LO_straddle_pos', 'LN_straddle_pos', 'CL_future_pos', 'NG_future_pos',
              'LO_option_cost', 'LN_option_cost', 'CL_future_cost', 'NG_future_cost']]
            .sum())
    
    summ.index = pd.to_datetime(summ.index)

    return summ

@nb.njit()
def cumargmax(s):
    """
    Helper function to find max drawdown
    """
    curr_i, curr_val = -1, -np.inf
    ret1 = np.empty(len(s), dtype=nb.uint64)
    ret2 = np.empty(len(s), dtype=nb.float64)
    for i, x in enumerate(s):
        if x > curr_val:
            curr_i, curr_val = i, x
        ret1[i], ret2[i] = curr_i, curr_val
    return ret1, ret2

def add_running_statistics(df):
    """
    Calculate cumulative summary and drawdown
    """
    df = df.copy()
    for p in ['Total_']:
        df[p + 'pnl_cum'] = df[p + 'pnl'].cumsum()
    highwater_ind, df['highwater_pnl'] = cumargmax(df.Total_pnl_cum.values)
    df['highwater_ts'] = df.index[highwater_ind]
    df['drawdown'] = df.highwater_pnl - df.Total_pnl_cum
    return df

def get_monthly_stats(df):
    df = df.assign(month=df.index.strftime('%Y/%m'))
    return {k: v for k, v in df.groupby('month')}

def get_scalar_stats(df):
    """
    Get overall statistics including sharpe and sortino
    """
    ret = {}
    for col in ['LO_option_pnl', 'LN_option_pnl', 'CL_future_pnl', 'NG_future_pnl', 'LO_pnl', 'LN_pnl', 'Total_pnl', \
                'LO_option_contracts', 'LN_option_contracts', 'CL_future_contracts', 'NG_future_contracts', \
                'LO_option_cost', 'LN_option_cost', 'CL_future_cost', 'NG_future_cost', 'LO_straddle_pos', 'LN_straddle_pos', \
                'LO_option_delta', 'LO_option_gamma', 'LO_option_theta', 'LO_option_vega']:
                
        ret[col] = df[col].sum()
    ret['Total_cost'] = ret['LO_option_cost'] + ret['LN_option_cost'] + ret['CL_future_cost'] + ret['NG_future_cost']
    ret['Total_pnl_before_cost'] = ret['Total_pnl'] + ret['Total_cost']
    ret['pnl_per_option_trade'] = ret['Total_pnl'] / (ret['LO_option_contracts'] + ret['LN_option_contracts'] + 1e-9)
    ret['max_LO_straddle_pos'] = df.LO_straddle_pos.abs().max().astype(int)
    ret['max_LN_straddle_pos'] = df.LN_straddle_pos.abs().max().astype(int)
    ret['max_drawdown'] = max(df.drawdown.max(), 0)
    ret['max_drawdown_pct'] = df.drawdown.max() / (1e-9 + ret['Total_pnl'])
    
    df['pnl'] = df['Total_pnl']
    m = df.pnl.mean()
    if m == 0:
        ret['sharpe'] = ret['sortino'] = 0
    else:
        ret['sharpe'] = m / df.pnl.std() * np.sqrt(252)
        ret['sortino'] = m / np.sqrt(((df.pnl[df.pnl < 0] - m)**2).mean()) * np.sqrt(252)
    return pd.Series(ret)

In [117]:
summ = summarize_bts(new_bts)
summ_stats = get_scalar_stats(add_running_statistics(summ))

In [119]:
def color_negative_red(val):
    if isinstance(val, float) or isinstance(val, int):
        color = 'red' if val < 0 else 'green'
    else:
        color = 'red' if val[0] == '-' else 'green'
    return 'color: %s' % color

# Visualition of params and backtest performance.
def mk_summ_table(bts, by_month=False):
    if by_month:
        table = pd.DataFrame({k: get_scalar_stats(add_running_statistics(v)) for k, v in get_monthly_stats(bts).items()}).T.sort_index()
        table.index.name = 'Month'
    else:
        table = get_scalar_stats(add_running_statistics(bts)).to_frame('Total').T
    table = table[['Total_pnl', 'LO_pnl', 'LN_pnl', 'Total_pnl_before_cost', 'Total_cost', 'LO_option_contracts', 'LN_option_contracts', 'max_LO_straddle_pos', 'max_LN_straddle_pos', 'pnl_per_option_trade', 'sharpe', 'sortino', 'max_drawdown']].copy()
    table = table.copy()
    table = table.dropna()
    
    for c in ['Total_pnl', 'LO_pnl', 'LN_pnl', 'Total_pnl_before_cost', 'Total_cost', 'max_drawdown']:
        table[c] = (table[c]/1000).apply(lambda x: '${:,.0f}k'.format(x) if x > 0 else '-${:,.0f}k'.format(abs(x)))
    
    for c in ['LO_option_contracts', 'LN_option_contracts', 'max_LO_straddle_pos', 'max_LN_straddle_pos']:# 'pnl_per_option_trade']:
        table[c] = table[c].astype(int)
    table['pnl_per_option_trade'] = round(table['pnl_per_option_trade'], 2)
        
    for c in ['sharpe', 'sortino']:
        table[c] = table[c].apply(lambda x: '{:,.2f}'.format(x))
        
    table.columns = pd.MultiIndex.from_tuples([
            ('P&L', 'PnL Total '),
            ('P&L', 'LO Total'),
            ('P&L', 'LN Total'),
            ('P&L', 'Before Cost'),
            ('Total Cost', '$'),
            ('Contracts Traded', 'LO Option'),
            ('Contracts Traded', 'LN Option'),
            ('Maximum Position', 'Max LO straddle Pos'),
            ('Maximum Position', 'Max LN straddle Pos'),
            ('P&L/Contract', '$'),
            ('Sharpe', ''),
            ('Sortino', ''),
            ('Max Drawdown', '$')]
        ) 
    cm = sns.light_palette("yellow", as_cmap=True)
    return table.style.background_gradient(cmap=cm, low=0.0).applymap(color_negative_red).render()

# PnL and other Backtest summary statistics by month

In [120]:
value = mk_summ_table(summ) + mk_summ_table(summ, by_month=True)
display(HTML(value))

# PnL & Option Straddle Position Curve

In [121]:
fig = figure(width=600, height=300, x_axis_type='datetime', y_axis_label='P&L ($)')
fig.yaxis.formatter = bom.NumeralTickFormatter(format='($0.0a)')
fig.line(x=summ.index, y=summ.LO_pnl.cumsum(), legend='LO P&L', color='red', alpha=0.3)
fig.line(x=summ.index, y=summ.LN_pnl.cumsum(), legend='LN P&L', color='blue', alpha=0.3)
fig.line(x=summ.index, y=summ.Total_pnl.cumsum(), legend='Total P&L', color='green', line_width=2)
fig.legend.location = 'top_left'

fig2 = figure(width=600, height=300, x_axis_type='datetime', y_axis_label='Straddle Position')
fig2.line(x=summ.index, y=summ.LO_straddle_pos, legend='LO Straddle Pos', color='purple', alpha=0.7)
fig2.line(x=summ.index, y=summ.LN_straddle_pos, legend='LN Straddle Pos', color='black', alpha=0.7)
fig2.legend.location = 'top_left'

show(gridplot([[fig], [fig2]]), notebook_handle=True)

# LO Option Greeks Curve (Delta Hedged in backtest)
## Maybe more comparable to use Cash greeks (Cash amount change per 1% future move) across different products, but subject to trader's preference

## All greeks are simpliefied to aggregate together assuming 100% correlation between different expiries. Ideally greeks analysis will be separated into different contracts or aggregated with calibrated correlation.

In [122]:
fig = figure(width=400, height=300, x_axis_type='datetime', y_axis_label='LO Option Delta')
fig.line(x=summ.index, y=summ.LO_option_delta, legend='LO option Delta (No. of futures)', color='red')

fig2 = figure(width=400, height=300, x_axis_type='datetime', y_axis_label='LO Option Gamma (per 1 future point move)')
fig2.line(x=summ.index, y=summ.LO_option_gamma, legend='LO option Gamma', color='green')

fig3 = figure(width=400, height=300, x_axis_type='datetime', y_axis_label='LO Option Vega')
fig3.line(x=summ.index, y=summ.LO_option_vega, legend='LO option Vega (per 1 point vol move)', color='blue')

fig4 = figure(width=400, height=300, x_axis_type='datetime', y_axis_label='LO Option Theta')
fig4.line(x=summ.index, y=summ.LO_option_theta, legend='LO option Theta', color='yellow')

show(gridplot([[fig, fig2], [fig3, fig4]]), notebook_handle=True)

# LN Option Greeks Curve (Delta Hedged in backtest)

In [123]:
fig = figure(width=400, height=300, x_axis_type='datetime', y_axis_label='LN Option Delta')
fig.line(x=summ.index, y=summ.LN_option_delta, legend='LN option Delta (No. of futures)', color='red')

fig2 = figure(width=400, height=300, x_axis_type='datetime', y_axis_label='LN Option Gamma (per 1 future point move)')
fig2.line(x=summ.index, y=summ.LN_option_gamma, legend='LN option Gamma', color='green')

fig3 = figure(width=400, height=300, x_axis_type='datetime', y_axis_label='LN Option Vega')
fig3.line(x=summ.index, y=summ.LN_option_vega, legend='LN option Vega (per 1 point vol move)', color='blue')

fig4 = figure(width=400, height=300, x_axis_type='datetime', y_axis_label='LN Option Theta')
fig4.line(x=summ.index, y=summ.LN_option_theta, legend='LN option Theta', color='yellow')

show(gridplot([[fig, fig2], [fig3, fig4]]), notebook_handle=True)